# 2.4 Local Model Deployment
# 2.4 Local Model Deployment
# 2.4 本地模型部署

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/forhow134/ai-coding-guide/blob/main/demos/02-ai-landscape/local_deployment.ipynb)

**预计 API 费用：$0**（完全本地运行）

**⚠️ 重要提示：** 本 Notebook 展示的是代码模式，但需要在本地安装并运行 Ollama。Colab 环境下无法直接运行 Ollama 服务。

本 Notebook 演示如何：
1. 通过 OpenAI 兼容 API 调用本地 Ollama 模型
2. 列出本地可用模型
3. 对比本地模型与云端模型的性能

## 前置准备：安装 Ollama

在运行本 Notebook 前，请先在本地安装 Ollama：

**macOS / Linux：**
```bash
curl -fsSL https://ollama.com/install.sh | sh
```

**Windows：**
前往 [Ollama 官网](https://ollama.com/download) 下载安装包。

**下载并运行模型：**
```bash
# 下载 Qwen2.5-7B（约 4.7GB）
ollama run qwen2.5:7b

# 或者下载 Llama 3.2-3B（约 2GB，更轻量）
ollama run llama3.2:3b
```

**启动 API 服务：**
```bash
ollama serve
```

默认会在 `http://localhost:11434` 启动 HTTP API。

## Install Dependencies
<!-- 安装依赖 -->

In [ ]:
!pip install -q openai

## 检查 Ollama 服务状态

确保 Ollama 服务正在运行。

In [ ]:
import requests

try:
    response = requests.get("http://localhost:11434/api/tags", timeout=5)
    if response.status_code == 200:
        print("✅ Ollama 服务运行正常")
        
        # 列出已下载的模型
        models = response.json().get("models", [])
        if models:
            print(f"\n已下载的模型（{len(models)} 个）：")
            for model in models:
                print(f"  - {model['name']} (大小: {model['size'] / 1e9:.2f} GB)")
        else:
            print("\n⚠️  未找到已下载的模型，请先运行：ollama run qwen2.5:7b")
    else:
        print(f"❌ Ollama 服务返回错误：{response.status_code}")
except requests.exceptions.ConnectionError:
    print("❌ 无法连接到 Ollama 服务")
    print("请确保：")
    print("  1. 已安装 Ollama（https://ollama.com/download）")
    print("  2. 已运行 'ollama serve'")
    print("  3. 已下载模型（如 'ollama run qwen2.5:7b'）")
except Exception as e:
    print(f"❌ 检查失败：{e}")

## Basics用法：用 OpenAI SDK Call本地Models
<!-- 基础用法：用 OpenAI SDK 调用本地模型 -->

Ollama 提供 OpenAI 兼容的 API，只需修改 `base_url` 即可。

In [ ]:
from openai import OpenAI

# 连接到本地 Ollama 服务
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama",  # Ollama 不验证 API key，随便填
)

# 调用本地模型
response = client.chat.completions.create(
    model="qwen2.5:7b",  # 模型名要和 ollama list 中的一致
    messages=[
        {"role": "system", "content": "你是一个专业的 Python 编程助手"},
        {"role": "user", "content": "用一句话解释什么是装饰器？"},
    ],
)

print("【Qwen2.5-7B 本地模型】")
print(response.choices[0].message.content)
print(f"\nTokens: {response.usage.total_tokens}")

## StreamingOutput：实时显示Generate内容
<!-- 流式输出：实时显示生成内容 -->

像 ChatGPT 一样逐字输出。

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama",
)

print("【流式输出】")
print("AI: ", end="", flush=True)

response = client.chat.completions.create(
    model="qwen2.5:7b",
    messages=[{"role": "user", "content": "用 50 字介绍 Python 的历史"}],
    stream=True,  # 启用流式输出
)

for chunk in response:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)

print("\n")

## 列出本地可用Models
<!-- 列出本地可用模型 -->

通过 Ollama API 查看已下载的模型。

In [ ]:
import requests

def list_local_models():
    """列出本地所有可用的 Ollama 模型"""
    try:
        response = requests.get("http://localhost:11434/api/tags")
        response.raise_for_status()
        
        models = response.json().get("models", [])
        
        if not models:
            print("⚠️  未找到已下载的模型")
            print("请运行：ollama run qwen2.5:7b")
            return []
        
        print(f"找到 {len(models)} 个本地模型：\n")
        
        for model in models:
            name = model["name"]
            size_gb = model["size"] / 1e9
            modified = model.get("modified_at", "未知")
            
            print(f"  📦 {name}")
            print(f"     大小: {size_gb:.2f} GB")
            print(f"     更新时间: {modified[:10]}")
            print()
        
        return [m["name"] for m in models]
    
    except requests.exceptions.ConnectionError:
        print("❌ 无法连接到 Ollama 服务")
        return []
    except Exception as e:
        print(f"❌ 查询失败：{e}")
        return []

available_models = list_local_models()

## 性能基准Test
<!-- 性能基准测试 -->

对比本地模型与云端模型的速度和质量。

In [ ]:
import time
from openai import OpenAI

def benchmark_model(client, model, question):
    """测试模型的速度和质量"""
    try:
        start = time.time()
        
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": question}],
        )
        
        latency = time.time() - start
        answer = response.choices[0].message.content
        tokens = response.usage.total_tokens
        
        return {
            "model": model,
            "latency": latency,
            "tokens": tokens,
            "answer": answer,
            "tokens_per_sec": tokens / latency,
            "success": True,
        }
    
    except Exception as e:
        return {
            "model": model,
            "error": str(e),
            "success": False,
        }

In [ ]:
# 本地模型客户端
local_client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama",
)

# Test问题
test_question = "用一句话解释什么是递归？"

print(f"测试问题：{test_question}\n")
print("=" * 80)

# Test本地模型
if available_models:
    local_model = available_models[0]  # 使用第一个可用模型
    print(f"\n【本地模型】{local_model}")
    
    local_result = benchmark_model(local_client, local_model, test_question)
    
    if local_result["success"]:
        print(f"回答：{local_result['answer']}")
        print(f"\n⏱️  延迟：{local_result['latency']:.2f}s")
        print(f"💰 Tokens: {local_result['tokens']}")
        print(f"⚡ 吞吐量：{local_result['tokens_per_sec']:.1f} tokens/s")
    else:
        print(f"❌ 测试失败：{local_result['error']}")
else:
    print("\n⚠️  未找到本地模型，跳过测试")

## Comparison云端Models（可选）
<!-- 对比云端模型（可选） -->

如果有 OpenAI API Key，可以对比本地与云端模型。

In [ ]:
import os
from getpass import getpass

# 可选：配置 OpenAI API Key 进行对比
if not os.environ.get("OPENAI_API_KEY"):
    print("提示：输入 OpenAI API Key 可以对比云端模型（按 Enter 跳过）")
    key = getpass("OpenAI API Key (可选): ")
    if key:
        os.environ["OPENAI_API_KEY"] = key

In [ ]:
if os.environ.get("OPENAI_API_KEY"):
    # 云端模型客户端
    cloud_client = OpenAI()
    
    print("\n【云端模型】GPT-4o-mini")
    
    cloud_result = benchmark_model(cloud_client, "gpt-4o-mini", test_question)
    
    if cloud_result["success"]:
        print(f"回答：{cloud_result['answer']}")
        print(f"\n⏱️  延迟：{cloud_result['latency']:.2f}s")
        print(f"💰 Tokens: {cloud_result['tokens']}")
        print(f"⚡ 吞吐量：{cloud_result['tokens_per_sec']:.1f} tokens/s")
        print(f"💵 估算成本：${(cloud_result['tokens'] * 0.15) / 1_000_000:.6f}")
    
    # 对比总结
    if available_models and local_result["success"] and cloud_result["success"]:
        print("\n" + "=" * 80)
        print("对比总结：")
        print("=" * 80)
        print(f"{'指标':<20s} {'本地模型':<25s} {'云端模型':<25s}")
        print("-" * 80)
        print(f"{'模型':<20s} {local_result['model']:<25s} {cloud_result['model']:<25s}")
        print(f"{'延迟':<20s} {local_result['latency']:.2f}s{'':<21s} {cloud_result['latency']:.2f}s")
        print(f"{'Tokens':<20s} {local_result['tokens']:<25d} {cloud_result['tokens']:<25d}")
        print(f"{'吞吐量':<20s} {local_result['tokens_per_sec']:.1f} tokens/s{'':<13s} {cloud_result['tokens_per_sec']:.1f} tokens/s")
        print(f"{'成本':<20s} {'$0（本地运行）':<25s} ${(cloud_result['tokens'] * 0.15) / 1_000_000:.6f}")
else:
    print("\n跳过云端模型对比（未配置 OPENAI_API_KEY）")

## 常用Models推荐
<!-- 常用模型推荐 -->

根据不同场景选择合适的模型：

| 模型 | 参数量 | 内存需求 | 擅长领域 | 下载命令 |
|-----|-------|---------|---------|----------|
| **qwen2.5:7b** | 7B | ~8GB | 中文、代码、通用 | `ollama run qwen2.5:7b` |
| **llama3.2:3b** | 3B | ~4GB | 英文、速度快、资源少 | `ollama run llama3.2:3b` |
| **deepseek-r1:7b** | 7B | ~8GB | 推理、数学、逻辑 | `ollama run deepseek-r1:7b` |
| **codellama:7b** | 7B | ~8GB | 代码生成、补全 | `ollama run codellama:7b` |
| **mistral:7b** | 7B | ~8GB | 多语言、通用 | `ollama run mistral:7b` |

## Hands-on Exercises
<!-- 动手练习 -->

1. 下载多个不同大小的模型（3B、7B、14B），对比它们的速度和质量
2. 尝试创建 Modelfile 自定义模型（添加系统提示词、调整参数）
3. 实现一个智能路由：根据问题复杂度自动选择本地模型或云端模型
4. 测试流式输出的逐字显示效果

## Summary
<!-- 小结 -->

- **本地部署的优势**：
  - ✅ 零 API 成本，适合大量调用
  - ✅ 数据隐私，不上传到云端
  - ✅ 离线可用，不依赖网络
  - ✅ 低延迟，适合实时交互

- **本地部署的劣势**：
  - ❌ 硬件要求：7B 模型需要 8GB+ 内存
  - ❌ 能力较弱：7B 模型弱于 GPT-4o / Claude Opus
  - ❌ 维护成本：需要手动更新模型

**关键要点：**
- Ollama 提供 OpenAI 兼容 API，只需改 `base_url`
- 推荐模型：Qwen2.5-7B（中文+代码）、Llama3.2-3B（轻量）
- 量化技术可减少 70% 内存占用
- 适合场景：数据隐私、成本控制、离线环境

下一步：深入学习 LLM 的工作原理 → [第 3 章：LLM 核心原理](../../docs/03-llm-fundamentals/index.md)